In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('/content/Churn.csv')
df

,Unnamed: 0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction Score,Card Type,Point Earned,Exited
0,0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,2,DIAMOND,464,Yes
1,1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,1,3,DIAMOND,456,No
2,2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3,DIAMOND,377,Yes
3,3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,5,GOLD,350,No
4,4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,5,GOLD,425,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0,1,DIAMOND,300,No
9996,9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,5,PLATINUM,771,No
9997,9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1,3,SILVER,564,Yes
9998,9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,2,GOLD,339,Yes


In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
df

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction Score,Card Type,Point Earned,Exited
0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,2,DIAMOND,464,Yes
1,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,1,3,DIAMOND,456,No
2,2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3,DIAMOND,377,Yes
3,3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,5,GOLD,350,No
4,4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,5,GOLD,425,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0,1,DIAMOND,300,No
9996,9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,5,PLATINUM,771,No
9997,9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1,3,SILVER,564,Yes
9998,9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,2,GOLD,339,Yes


In [ ]:
df[df['Balance'] == 0]['Exited'].value_counts()

Exited
No     3117
Yes     500
Name: count, dtype: int64

In [ ]:
X=df.drop('Exited',axis=1)
y=df['Exited'].copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

num_features = X_train.select_dtypes(include = np.number).columns
cat_features = X_train.select_dtypes(include = ['object']).columns
cat_features


Index(['Geography', 'Gender', 'Card Type'], dtype='object')

In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,PolynomialFeatures
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from sklearn.metrics import classification_report

In [ ]:
np.random.seed(42)

num_pipeline = make_pipeline(SimpleImputer(strategy = 'median'),
                             StandardScaler(),
                             PolynomialFeatures(degree = 2, include_bias = False))

cat_pipeline = make_pipeline(SimpleImputer(strategy = 'most_frequent'),
                             OneHotEncoder(handle_unknown='ignore'))

transformer = ColumnTransformer([
    ('num_pipeline', num_pipeline, num_features),
    ('cat_pipeline', cat_pipeline, cat_features)
], remainder = 'passthrough')

In [ ]:
pipeline = make_imb_pipeline(
    transformer,
    SMOTE(random_state=42),
    LogisticRegression()
)

In [ ]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(include_bias=False))]),
                                                  Index(['Unnamed: 0', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts',...
       'Complain', 'Satisfaction Score', 'Point Earned'],
      dtype='object')),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Geography', 'Gender', 'Card Type'], dtype='object'))])),
                ('smote', SMOTE(random_state=42)),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00      1607
         Yes       0.99      0.99      0.99       393

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
pipeline.score(X_train, y_train)

0.99925

In [ ]:
pipeline = make_pipeline(
    transformer,
    RandomForestClassifier()
)

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(include_bias=False))]),
                                                  Index(['Unnamed: 0', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts',...ember', 'EstimatedSalary',
       'Complain', 'Satisfaction Score', 'Point Earned'],
      dtype='object')),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Geography', 'Gender', 'Card Type'], dtype='object'))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
pipeline.score(X_test, y_test)


0.999

In [ ]:
from sklearn.metrics import f1_score
y_pred = pipeline.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')

In [ ]:
np.random.seed(42)

forest_pipeline = make_pipeline(
    transformer,
    RandomForestClassifier()
)

forest_pipeline.fit(X_train, y_train)
forest_pipeline.score(X_test, y_test)

0.999

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bgg_pipeline = make_pipeline(
   transformer,
    BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                      n_estimators=500, max_samples=100,
                      n_jobs=-1, random_state=42)
)


In [ ]:
bgg_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(include_bias=False))]),
                                                  Index(['Unnamed: 0', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts',...
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Geography', 'Gender', 'Card Type'], dtype='object'))])),
                ('baggingclassifier',
                 BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                   max_samples=100, n_estimators=500, n_jobs=-1,
                                   random_state=42))])

In [ ]:
bgg_pipeline.score(X_test,y_test)

0.999

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_pipeline=make_pipeline(transformer,
                           AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=30,
    learning_rate=0.5,random_state=42))
ada_pipeline.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler()),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(include_bias=False))]),
                                                  Index(['Unnamed: 0', 'CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts',...
      dtype='object')),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Geography', 'Gender', 'Card Type'], dtype='object'))])),
                ('adaboostclassifier',
                 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                                    learning_rate=0.5, n_estimators=30,
                                    random_state=42))])

In [ ]:
ada_pipeline.score(X_test,y_test)

0.999